In [ ]:
%load_ext autoreload
%autoreload 2

# parquet

> Binary storage formats.

## Apache Parquet

File format for data storage. Provides efficient data compression and encoding schemes with enhanced performance to handle complex data in bulk. 

> Apache Parquet is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language.

[Website](https://parquet.apache.org/)

[Wikipedia](https://en.wikipedia.org/wiki/Apache_Parquet)

[Parquet vs CSV: AWS processing costs](https://blog.openbridge.com/how-to-be-a-hero-with-powerful-parquet-google-and-amazon-f2ae0f35ee04)

[Docs](https://parquet.apache.org/documentation/latest/)

Features:

- columnar storage, only read the data of interest
- efficient binary packing
- choice of compression algorithms and encoding
- split data into files, allowing for parallel processing
- range of logical types
- statistics stored in metadata allow for skipping unneeded chunks
- data partitioning using the directory structure

Hierarchically, a file consists of one or more row groups. A row group contains exactly one column chunk per column. Column chunks contain one or more pages. 

## fastparquet

Python implementation of the *Apache Parquet* format.
Part of *dask* ecosystem, designed to work well with dask for parallel execution.

Latest release: 0.3.3

Not all parts of the Parquet-format have been implemented yet or tested. 
Not all output options will be compatible with every other Parquet framework, which each implement only a subset of the standard.
Usage decisions: writing parquet files that are compatible with other parquet implementations, versus performance when writing data for reading back with fastparquet.

[GitHub page](https://github.com/dask/fastparquet)

[Docs](https://fastparquet.readthedocs.io/en/latest/)

### Variations

- uncompressed, gzip, snappy (install `python-snappy` from conda-forge separately)

### Issues

- As of 0.3.3, pandas extended dtypes like "Int64" are not supported, but there is [wip](https://github.com/dask/fastparquet/pull/483)

In [ ]:
import pandas as pd
import fastparquet as fp

import ig_format
from ig_format import pandas as igpd

In [ ]:
data_dir = './out/extracts/100k'
schema_path = './out/schema.json'
data_years = range(1997, 2000)

dt = igpd.dtypes_from_schema(schema_path)
df = pd.read_csv(data_dir + '/2000.csv', dtype=dt)

In [ ]:
# convert nullable ints to floats for fastparquet compatibility
# could be part of schema creation
for c in df:
    if isinstance(df[c].dtype, pd.Int64Dtype):
        df[c] = df[c].astype('float64')

fp.write('./tmp/2000.parquet', df, compression='gzip')

In [ ]:
pf = fp.ParquetFile('./tmp/2000.parquet')
dfp = pf.to_pandas(['company', 'abi', 'state', 'naics', 'employees'])
dfp.head()

## Conversion from SAS

### pyreadstat

https://github.com/Roche/pyreadstat

- wrapped C library, faster than sas7bdat and pd.read_sas
- available from conda-forge
- supports SAS, Stata and SPSS formats
- read meta, read column subset, read row chunks